In [ ]:
!pip3 install indic-nlp-library

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch

In [ ]:
from indicnlp.tokenize import indic_tokenize

In [ ]:
data = pd.read_csv(
    "/kaggle/input/language-translation1/Sentence pairs in English-Hindi - 2025-02-11.tsv",
    sep="\t",
    header=None,
    names=["SrcSennId","SrcSent","DatSentID","DatSent"]
)

In [ ]:
data.head()

In [ ]:
# data.drop(labels=[data.columns[0], data.columns[2]], axis=1, inplace=True)
# Save column names first
cols_to_drop = [data.columns[0], data.columns[2]]

# Drop
data.drop(columns=cols_to_drop, inplace=True)

In [ ]:
from transformers import AutoTokenizer

src_sent_tokenizer = AutoTokenizer.from_pretrained("t5-base")


In [ ]:
data["SrcSent"] = data["SrcSent"].apply(lambda x: src_sent_tokenizer.tokenize(x))


In [ ]:
data.head()

In [ ]:
# # data["DstSent"]= data["DstSent"].apply(lambda x: indic_tokenize.trivial_tokenize(x,lang="hi")
# data["DstSent"] = data["DstSent"].apply(
#     lambda x: indic_tokenize.trivial_tokenize(x, lang="hi")
# )
from indicnlp.tokenize import indic_tokenize

data["DatSent"] = data["DatSent"].apply(
    lambda x: indic_tokenize.trivial_tokenize(x, lang="hi")
)


In [ ]:
print(data.columns)


In [ ]:
# data["SrcSent"]= data["srcSent"].apply(arc_sent_tokenizer.convert_tokens_to_ids)
data["SrcSent_ids"] = data["SrcSent"].apply(
    lambda tokens: src_sent_tokenizer.convert_tokens_to_ids(tokens)
)



In [ ]:
Vs = src_sent_tokenizer.get_vocab()


In [ ]:
# Vd = set()
# for tokenized_hindi_sent in data["DatSent"]:
#     hindi_vocab.update(tokenized_hindi_sent)

Vd = set()
for sent in data["DatSent"]:
    # Convert to string (avoid NaN, lists, etc.)
    sent = str(sent)
    tokens = tokenizer.tokenize(sent)
    Vd.update(tokens)

print("Unique tokens in dataset:", len(Vd))
print(list(Vd)[:20])



In [ ]:
hindi_vocab = set()

for tokenized_hindi_sent in data["DatSent"]:
    hindi_vocab.update(tokenized_hindi_sent)


In [ ]:
Vd = dict()

# Special tokens
Vd["<PAD>"] = 0
Vd["<SOS>"] = 1
Vd["<EOS>"] = 2

# Add rest of vocab
for idx, token in enumerate(hindi_vocab, start=3):
    Vd[token] = idx


In [ ]:
def convert_hindi_token_to_ids(hindi_sent):
    return[Vd[token] for token in hindi_sent]
    

In [ ]:
data["DatSent"] = data["DatSent"].apply(
    lambda x: convert_hindi_token_to_ids(x)
)


In [ ]:
def insert_sos_token_id(hindi_sent_token_ids):
    return [1] + hindi_sent_token_ids


In [ ]:
data["DatSentInput"] = data["DatSent"].apply(lambda x: insert_sos_token_id(x))


In [ ]:
data.head()

In [ ]:
def insert_eos_token_id(hindi_sent_token_ids):
    return hindi_sent_token_ids + [2]


In [ ]:
data["DatSentLavel"] = data["DatSent"].apply(
    lambda x: insert_eos_token_id(x)
)


In [ ]:
data.head()


In [ ]:
data.drop(labels=[data.columns[1]], axis=1, inplace=True)


In [ ]:
print(data.columns)


In [ ]:
X = list(data["SrcSent"])
Y_input = list(data["DatSentInput"])
Y_label = list(data["DatSentLavel"])


In [ ]:
X_tensor = [torch.tensor(ids) for ids in data["SrcSent_ids"]]
Y_input_tensor = [torch.tensor(ids) for ids in data["DatSentInput"]]  # if these are already token IDs
Y_label_tensor = [torch.tensor(ids) for ids in data["DatSentLavel"]]   # if these are already token IDs


In [ ]:
X_padded = torch.nn.utils.rnn.pad_sequence(X_tensor,batch_first=True)
Y_padded_input = torch.nn.utils.rnn.pad_sequence(Y_input_tensor,batch_first=True)
Y_padded_label = torch.nn.utils.rnn.pad_sequence(Y_label_tensor,batch_first=True)

In [ ]:

Ns = X_padded.shape[1]
Nd = Y_padded_label.shape[1]

In [ ]:
class Encoder(torch.nn.Module):

    def __init__(self,src_lang_vocab_size,word_embedding_dim):
        super(Encoder,self).__init__()
        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=src_lang_vocab_size,
                                                       embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=word_embedding_dim,
                                               hidden_size=word_embedding_dim,
                                              batch_first=True)

    def forward(self,X_padded_mini_batch):

        first_embedding_layer_out = self.first_embedding_layer(X_padded_mini_batch)
        encoder_output, (final_encoder_output,final_cell_state) = self.second_lstm_layer(first_embedding_layer_out)

        return encoder_output, (final_encoder_output,final_cell_state)

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self,dst_lang_vocab_size,word_embedding_dim):
        super(Decoder,self).__init__()

        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=dst_lang_vocab_size,
                                                       embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=word_embedding_dim,
                                               hidden_size=word_embedding_dim,
                                              batch_first=True)
        self.prediction_layer = torch.nn.Linear(in_features=words_embedding_dim,out_features=dst_lang_vocab_size)
        self.prediction_layer_activation = torch.nn.Softmax(dim=2)

    def forward(self,Y_padded_input_mini_batch,final_encoder_output,final_cell_state):

        first_embedding_layer_out = self.first_embedding_layer(Y_padded_input_mini_batch)
        decoder_lstm_layer_out, (final_decoder_lstm_layer_out, final_cell_state) = self.second_lstm_layer(first_embedding_layer_out,
                                                                                                         final_encoder_output,
                                                                                                          final_cell_state)
        prdiction=self.prediction_layer_activation.prediction(decoder)
        return decoder_lstm_layer_out, (final_decoder_lstm_layer_out, final_cell_state)